In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 91.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 78.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 52.3 MB/s eta 0:00:00


In [ ]:
import json

data = []
with open('/content/validation_data.jsonl', 'r') as file:
    for line in file:
        data.append(json.loads(line))

In [ ]:
text = [d["paragraph"] for d in data]

In [ ]:
text[0]

'وبعد أن ألقينا الضوء على أهم فتوحات بلاد الشام تعال معا نتناول بيت المقدس بوصفه نموذجا لفتوحات هذه البلاد بشيء من التفصيل .\nفتح بيت المقدس ١٥هـ / ٦٣٦م:\nلحق عمرو بن العاص بالروم، وشدد عليهم الحصار ، ولم يتمكن الروم من فكه، وهرب الأرطبون إلى مصر مركز ثقل الوجود البيزنطي في إفريقيا لصد هجمات المسلمين.\nوأثناء الحصار كانت تجرى محادثات لدخول أهل بيت المقدس في الصلح مع المسلمين، فلم يجد بطريرك بيت المقدس أمامه سوى المصالحة، ولكنه اشترط أن يتم الصلح مع الخليفة عمر بن الخطاب بنفسه وذلك لما للمدينة من قدسية وأهمية دينية كبيرة، فأرسل عمرو بن العاص بهذه الأخبار إلى الخليفة بالمدينة النبوية، وقال له في رسالته: "إني أعالج حربا كؤدًا صدومًا وبلادا ادخرت لك فلك رأيك" فكان أول خروج للخليفة من المدينة بعد موت رسول الله ، هذا الخروج الأجل لكي يفتح بيت المقدس . وكان أول نزول له بمدن الشام بالجابية، والتي وفدت عليه فيها رسل البطريرك طالبين الصلح، فصالحهم عمر صلحا لم تحظ به مدينة أخرى في البلاد المفتوحة.... \nهل تعرف شروط هذا الصلح ؟\nاقرأ النص التالى لتتعرف كيف كان الخليفة عمر سخيا في منحه الامتيازات ل

In [ ]:
from transformers import AutoTokenizer, AutoModelWithLMHead
import torch
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
m_name = "marefa-nlp/summarization-arabic-english-news"

tokenizer = AutoTokenizer.from_pretrained(m_name)
model = AutoModelWithLMHead.from_pretrained(m_name).to(device)

def get_summary(text, tokenizer, model, device="cpu", num_beams=2):
    if len(text.strip()) < 50:
        return ["Please provide more longer text"]

    text = "summarize: <paragraph> " + " <paragraph> ".join([ s.strip() for s in sent_tokenize(text) if s.strip() != ""]) + " </s>"
    text = text.strip().replace("\n","")

    tokenized_text = tokenizer.encode(text, return_tensors="pt").to(device)

    summary_ids = model.generate(
            tokenized_text,
            max_length=512,
            num_beams=num_beams,
            repetition_penalty=1.5,
            length_penalty=1.0,
            early_stopping=True
    )

    output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return [ s.strip() for s in output.split("<hl>") if s.strip() != "" ]


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1362: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [ ]:
## Prepare Samples

samples = text

# Get summaries
for i in samples:
  print("Paragraph no.: ", samples.index(i))
  print("Original Article:", i)
  print("\n===========\nSummary: \n")
  hls = get_summary(i, tokenizer, model, device)
  for hl in hls:
      print("\t-", hl)

Paragraph no.:  0
Original Article: وبعد أن ألقينا الضوء على أهم فتوحات بلاد الشام تعال معا نتناول بيت المقدس بوصفه نموذجا لفتوحات هذه البلاد بشيء من التفصيل .
فتح بيت المقدس ١٥هـ / ٦٣٦م:
لحق عمرو بن العاص بالروم، وشدد عليهم الحصار ، ولم يتمكن الروم من فكه، وهرب الأرطبون إلى مصر مركز ثقل الوجود البيزنطي في إفريقيا لصد هجمات المسلمين.
وأثناء الحصار كانت تجرى محادثات لدخول أهل بيت المقدس في الصلح مع المسلمين، فلم يجد بطريرك بيت المقدس أمامه سوى المصالحة، ولكنه اشترط أن يتم الصلح مع الخليفة عمر بن الخطاب بنفسه وذلك لما للمدينة من قدسية وأهمية دينية كبيرة، فأرسل عمرو بن العاص بهذه الأخبار إلى الخليفة بالمدينة النبوية، وقال له في رسالته: "إني أعالج حربا كؤدًا صدومًا وبلادا ادخرت لك فلك رأيك" فكان أول خروج للخليفة من المدينة بعد موت رسول الله ، هذا الخروج الأجل لكي يفتح بيت المقدس . وكان أول نزول له بمدن الشام بالجابية، والتي وفدت عليه فيها رسل البطريرك طالبين الصلح، فصالحهم عمر صلحا لم تحظ به مدينة أخرى في البلاد المفتوحة.... 
هل تعرف شروط هذا الصلح ؟
اقرأ النص التالى لتتعرف كيف كان الخليفة